In [40]:
import os
import numpy as np
import pandas as pd
from glob import glob
pd.options.display.max_columns = 999

folder_location = glob('C:\\Users\limzi\OneDrive\Forecasting & Reporting\Jeff Files\PowerBi Files\pcsg_shopify\*.csv')

pcsg_shopify = pd.concat([pd.read_csv(x, parse_dates=['Paid at', 'Created at'], date_parser=lambda x: pd.to_datetime(x.rpartition('+')[0]), dtype={'Lineitem sku': 'str', 'Name': 'str'}) for x in folder_location] ,ignore_index=True)

# len(pcsg_shopify)
# pcsg_shopify.drop_duplicates(inplace = True)
# pcsg_shopify.drop_duplicates(inplace = True)
# len(pcsg_shopify)

keep_columns = ['Name', 'Email','Paid at', 'Subtotal', 'Shipping', 'Taxes', 'Total', 'Discount Code', 'Discount Amount','Created at', 'Lineitem quantity', 'Lineitem name', 'Lineitem price', 'Lineitem compare at price', 'Lineitem sku','Lineitem discount']

pcsg_shopify_simple = pcsg_shopify[keep_columns].sort_values('Name', ascending = False).drop_duplicates()

# len(pcsg_shopify_simple)

sku_master = pd.read_excel("C:/Users/limzi/OneDrive/Forecasting & Reporting/PCSG Master List.xlsx", sheet_name='SKU List' )

# Import bundle sheet of masterlist file
bundle_master = pd.read_excel("C:/Users/limzi/OneDrive/Forecasting & Reporting/PCSG Master List.xlsx",
                              sheet_name='Bundles').rename(columns = {'Parent SKU': "parent_sku",
                                                                      'Parent Name': 'parent_name',
                                                                      'Quantity': 'child_quantity',
                                                                      'Child SKU': "child_sku",
                                                                      'Child Name': 'child_name'}).astype({'child_sku': 'str',
                                                                                                           'parent_sku': 'str'})
# Use (how = "Left") in order to duplicate the line item information on each composite SKU
pcsg_shopify_simple = pd.merge(pcsg_shopify_simple, bundle_master, left_on="Lineitem sku", right_on = 'parent_sku', how='left')

pcsg_shopify_simple['parent_sku'].fillna(pcsg_shopify_simple['Lineitem sku'], inplace=True)
pcsg_shopify_simple['child_sku'].fillna(pcsg_shopify_simple['Lineitem sku'], inplace=True)


# pcsg_shopify_merge['total_quantity'].fillna(pcsg_shopify_merge['Lineitem quantity'], inplace = True)

# Calculates the actual quantity of composite SKUs sold (since, e.g., there might be more than one 2010s sold in a bundle)
pcsg_shopify_simple['child_subtotal_quantity'] =  pcsg_shopify_simple['child_quantity'] * pcsg_shopify_simple['Lineitem quantity']
# same concept, repeat the quantities with individual SKUs in the child_quantity column
pcsg_shopify_simple['child_subtotal_quantity'].fillna(pcsg_shopify_simple['Lineitem quantity'], inplace=True)

pcsg_shopify_simple['bundle_status'] = pcsg_shopify_simple['parent_name'].apply(lambda x: 0 if pd.isna(x) else 1)

# keep_merge_columns = ['Name', 'Email', 'Paid at', 'Subtotal', 'Shipping', 'Taxes', 'Total', 'Discount Code', 'Discount Amount', 'Created at', 'Lineitem quantity', 'Lineitem name', 'Lineitem price', 'Lineitem compare at price', 'Lineitem sku', 'Lineitem discount', 'parent_sku', 'parent_name', 'child_sku', 'child_name', 'child_quantity',  'child_subtotal_quantity', 'bundle_status']
# pcsg_shopify_simple.to_csv('C:\\Users\limzi\OneDrive\Forecasting & Reporting\Jeff Files\PowerBi Files\pcsg_shopify_debundled\pcsg_shopify_orders_debundled.csv')


In [41]:
def replace_bundle_products(x):
    if (x['Lineitem compare at price'] == 0 or np.isnan(x['Lineitem compare at price'])):
        # return x['Lineitem price']
        if np.isnan(x['Price']):
            return x['Lineitem price']
        else: 
            return x['Price']
    else: 
        return x['Lineitem compare at price']

In [42]:
# pcsg_shopify_simple['Lineitem compare at price'] = pcsg_shopify_simple.apply(lambda x: x['Lineitem price'] if (x['Lineitem compare at price'] == 0 or np.isnan(x['Lineitem compare at price'])) else x['Lineitem compare at price'], axis = 1)
pcsg_shopify_simple['Lineitem compare at price'] = pcsg_shopify_simple.apply(replace_bundle_products, axis = 1)

In [44]:
# type(pcsg_shopify_simple['Lineitem price'][1])
# np.isnan(pcsg_shopify_simple['Lineitem compare at price'][1])
pcsg_shopify_simple['Lineitem price'] = pcsg_shopify_simple.apply(lambda x: x['Lineitem price'] if np.isnan(x['Discount Unit Price']) else x['Discount Unit Price'], axis = 1)

In [45]:
# pcsg_shopify_simple.loc[5]
# np.isnan(pcsg_shopify_simple['Discount Unit Price'][48])
pcsg_shopify_simple[pcsg_shopify_simple['bundle_status'] == 1]

,Name,Email,Paid at,Subtotal,Shipping,Taxes,Total,Discount Code,Discount Amount,Created at,Lineitem quantity,Lineitem name,Lineitem price,Lineitem compare at price,Lineitem sku,Lineitem discount,parent_sku,parent_name,child_sku,child_name,child_quantity,Unit Price,Price,Component Retail,Bundle Retail,% Discount,Discount Unit Price,child_subtotal_quantity,bundle_status
48,191569923831,leona@leonamay.com,NaT,NaN,NaN,NaN,NaN,NaN,NaN,2018-10-15 07:46:54,1,Pore Clarifying Charcoal Gel Mask - Sachet Bundle,5.000000,7.5,2657,0.0,2657,Pore Clarifying Charcoal Gel Mask,2659,Pore Clarifying Charcoal Gel Mask - Sachet,5.0,1.5,7.5,7.5,5.0,0.333333,5.000000,5.0,1
149,191569923772,anqi.la@hotmail.com,NaT,NaN,NaN,NaN,NaN,NaN,NaN,2018-10-13 11:18:40,2,Clear Acne Regular Strength Travel Kit,5.948718,8.0,4707,0.0,4707,Clear Regular Strength Trial Kit,6006,Clear Acne Face Wash - 30 ml,1.0,8.0,8.0,39.0,29.0,0.256410,5.948718,2.0,1
150,191569923772,anqi.la@hotmail.com,NaT,NaN,NaN,NaN,NaN,NaN,NaN,2018-10-13 11:18:40,2,Clear Acne Regular Strength Travel Kit,10.410256,14.0,4707,0.0,4707,Clear Regular Strength Trial Kit,6107,Clear Acne Regular Strength Cream 2.5% BP - 14 ml,1.0,14.0,14.0,39.0,29.0,0.256410,10.410256,2.0,1
151,191569923772,anqi.la@hotmail.com,NaT,NaN,NaN,NaN,NaN,NaN,NaN,2018-10-13 11:18:40,2,Clear Acne Regular Strength Travel Kit,12.641026,17.0,4707,0.0,4707,Clear Regular Strength Trial Kit,6207,Clear Acne Regular Strength Exfoliating Treatm...,1.0,17.0,17.0,39.0,29.0,0.256410,12.641026,2.0,1
202,191569923752,alyssacmy@gmail.com,2018-10-12 21:15:51,105.0,0.0,0.0,105.0,NaN,0.0,2018-10-12 21:15:50,1,Clear Acne Regular Strength Travel Kit,5.948718,8.0,4707,0.0,4707,Clear Regular Strength Trial Kit,6006,Clear Acne Face Wash - 30 ml,1.0,8.0,8.0,39.0,29.0,0.256410,5.948718,1.0,1
203,191569923752,alyssacmy@gmail.com,2018-10-12 21:15:51,105.0,0.0,0.0,105.0,NaN,0.0,2018-10-12 21:15:50,1,Clear Acne Regular Strength Travel Kit,10.410256,14.0,4707,0.0,4707,Clear Regular Strength Trial Kit,6107,Clear Acne Regular Strength Cream 2.5% BP - 14 ml,1.0,14.0,14.0,39.0,29.0,0.256410,10.410256,1.0,1
204,191569923752,alyssacmy@gmail.com,2018-10-12 21:15:51,105.0,0.0,0.0,105.0,NaN,0.0,2018-10-12 21:15:50,1,Clear Acne Regular Strength Travel Kit,12.641026,17.0,4707,0.0,4707,Clear Regular Strength Trial Kit,6207,Clear Acne Regular Strength Exfoliating Treatm...,1.0,17.0,17.0,39.0,29.0,0.256410,12.641026,1.0,1
236,191569923735,rina.ycs@gmail.com,2018-10-12 17:20:55,19.0,0.0,0.0,19.0,3b10acbb246e,10.0,2018-10-12 17:20:55,1,Clear Acne Regular Strength Travel Kit,5.948718,8.0,4707,0.0,4707,Clear Regular Strength Trial Kit,6006,Clear Acne Face Wash - 30 ml,1.0,8.0,8.0,39.0,29.0,0.256410,5.948718,1.0,1
237,191569923735,rina.ycs@gmail.com,2018-10-12 17:20:55,19.0,0.0,0.0,19.0,3b10acbb246e,10.0,2018-10-12 17:20:55,1,Clear Acne Regular Strength Travel Kit,10.410256,14.0,4707,0.0,4707,Clear Regular Strength Trial Kit,6107,Clear Acne Regular Strength Cream 2.5% BP - 14 ml,1.0,14.0,14.0,39.0,29.0,0.256410,10.410256,1.0,1
238,191569923735,rina.ycs@gmail.com,2018-10-12 17:20:55,19.0,0.0,0.0,19.0,3b10acbb246e,10.0,2018-10-12 17:20:55,1,Clear Acne Regular Strength Travel Kit,12.641026,17.0,4707,0.0,4707,Clear Regular Strength Trial Kit,6207,Clear Acne Regular Strength Exfoliating Treatm...,1.0,17.0,17.0,39.0,29.0,0.256410,12.641026,1.0,1
